In [1]:
########################################################################################################################
## -- libraries and packages -- ########################################################################################
########################################################################################################################
import os
import sys
sys.path.append(os.path.abspath(".."))
import torch
import transformer
from torch.utils.data import DataLoader

########################################################################################################################
## -- testing the data handler module -- ###############################################################################
########################################################################################################################
src_vocab_path = "../data/vocabs/en_vocab.json"
tgt_vocab_path = "../data/vocabs/fa_vocab.json"
src_path, src_name = "../data/dataset/Tatoeba.zip", "en.txt"
tgt_path, tgt_name = "../data/dataset/Tatoeba.zip", "fa.txt"
SOS_TOKEN, PAD_TOKEN, EOS_TOKEN = '<SOS>', '<PAD>', '<EOS>'

data_handler = transformer.DataHandler(src_path, src_name, src_vocab_path, tgt_path, tgt_name, tgt_vocab_path, 
                                       SOS_TOKEN, PAD_TOKEN, EOS_TOKEN, max_sequence_length = 256, max_sentences = 1000)

data = data_handler.data()

print(f"The DataHandler module preprocesses the data, and provides us with all properties we require: ")
print(f"Total Sentence Count After Validation: {len(data.src_sentences)}")
print(f"English (EN) sentence from dataset: \n{data.src_sentences[10]}")
print(f"Persian (FA) sentence from dataset: \n{data.tgt_sentences[10]}")

The DataHandler module preprocesses the data, and provides us with all properties we require: 
Total Sentence Count After Validation: 948
English (EN) sentence from dataset: 
i don't speak japanese.
Persian (FA) sentence from dataset: 
من ژاپنی صحبت نمی‌کنم.


In [2]:
########################################################################################################################
## -- testing the transformer dataset module -- ########################################################################
########################################################################################################################
batch_size = 32
dataset = transformer.TransformerDataset(data.src_sentences, data.tgt_sentences)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = False)

for batch_idx, (en_batch, fa_batch) in enumerate(dataloader):
  print(f"In batch {batch_idx}, there are {batch_size} English and Persian sentences")
  print(f"The first English sentence in this batch is: \n{en_batch[0]}")
  print(f"The first Persian sentence in this batch is: \n{fa_batch[0]}")
  break

In batch 0, there are 32 English and Persian sentences
The first English sentence in this batch is: 
i just don't know what to say.
The first Persian sentence in this batch is: 
من فقط نمی دانم چه بگویم.


In [3]:
########################################################################################################################
## -- testing the mask generator module -- #############################################################################
########################################################################################################################
mask_gen = transformer.MaskGenerator(max_sequence_length = 7)
en_test_batch = ("Hi",)
fa_test_batch = ("سلام",)
torch.set_printoptions(precision = 1)
enc_mask, dec_mask, dec_cross_mask = mask_gen.generate_masks(en_test_batch, fa_test_batch)
print(f"Encoder Padding Mask (shape: {enc_mask.shape}):")
print(enc_mask, end = "\n\n")
print(f"Decoder Padding + Look-Ahead Mask (shape: {dec_mask.shape}):")
print(dec_mask, end = "\n\n")
print(f"Decoder Cross Attention Mask (shape: {dec_cross_mask.shape}):")
print(dec_cross_mask)

Encoder Padding Mask (shape: torch.Size([1, 7, 7])):
tensor([[[ 0.0e+00,  0.0e+00, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [ 0.0e+00,  0.0e+00, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [-1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [-1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [-1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [-1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [-1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09]]])

Decoder Padding + Look-Ahead Mask (shape: torch.Size([1, 7, 7])):
tensor([[[ 0.0e+00, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [ 0.0e+00,  0.0e+00, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [ 0.0e+00,  0.0e+00,  0.0e+00, -1.0e+09, -1.0e+09, -1.0e+09, -1.0e+09],
         [ 0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00, -1.0e+09, -1.0e+09